In [1]:
import numpy as np 
import pandas as pd 
from scipy import signal
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# load the dataset
with pd.HDFStore("./data/train.h5", "r") as train:
    df = train.get("train")

In [3]:
df.head()

,id,timestamp,derived_0,derived_1,derived_2,derived_3,derived_4,fundamental_0,fundamental_1,fundamental_2,...,technical_36,technical_37,technical_38,technical_39,technical_40,technical_41,technical_42,technical_43,technical_44,y
0,10,0,0.370326,-0.006316,0.222831,-0.213030,0.729277,-0.335633,0.113292,1.621238,...,0.775208,NaN,NaN,NaN,-0.414776,NaN,NaN,-2.0,NaN,-0.011753
1,11,0,0.014765,-0.038064,-0.017425,0.320652,-0.034134,0.004413,0.114285,-0.210185,...,0.025590,NaN,NaN,NaN,-0.273607,NaN,NaN,-2.0,NaN,-0.001240
2,12,0,-0.010622,-0.050577,3.379575,-0.157525,-0.068550,-0.155937,1.219439,-0.764516,...,0.151881,NaN,NaN,NaN,-0.175710,NaN,NaN,-2.0,NaN,-0.020940
3,25,0,NaN,NaN,NaN,NaN,NaN,0.178495,NaN,-0.007262,...,1.035936,NaN,NaN,NaN,-0.211506,NaN,NaN,-2.0,NaN,-0.015959
4,26,0,0.176693,-0.025284,-0.057680,0.015100,0.180894,0.139445,-0.125687,-0.018707,...,0.630232,NaN,NaN,NaN,-0.001957,NaN,NaN,0.0,NaN,-0.007338


In [4]:
# classes of cols
list(set([c.split('_')[0] for c in df.columns]))

[u'technical', u'timestamp', u'derived', u'fundamental', u'y', u'id']

In [5]:
#print stats of all columns in the dataset
df.describe()

,id,timestamp,derived_0,derived_1,derived_2,derived_3,derived_4,fundamental_0,fundamental_1,fundamental_2,...,technical_36,technical_37,technical_38,technical_39,technical_40,technical_41,technical_42,technical_43,technical_44,y
count,1.710756e+06,1.710756e+06,1.637797e+06,1.629727e+06,1.312105e+06,1.561285e+06,1.304298e+06,1.686809e+06,1.031686e+06,1.341916e+06,...,1.708204e+06,1.691591e+06,1.691591e+06,1.690740e+06,1.708520e+06,1.666567e+06,1.690755e+06,1.706070e+06,1.473977e+06,1.710756e+06
mean,1.093858e+03,9.456257e+02,-4.536046e+00,7.729436e+11,-3.320328e-01,-5.046012e-01,1.801661e+01,-2.040938e-02,-5.703754e+08,-1.622954e-01,...,-8.584833e-02,-9.103397e-02,-8.156685e-02,-7.287001e-02,4.908321e-02,5.236218e-03,-1.699966e-02,-9.735299e-01,3.881475e-04,2.217509e-04
std,6.308563e+02,5.195685e+02,2.497382e+02,7.620606e+13,6.519810e+01,1.020749e+02,9.258360e+02,2.494859e-01,7.502322e+10,3.668150e+00,...,6.125852e-01,2.471038e-01,2.346534e-01,2.235729e-01,3.102316e-01,1.133733e-01,2.116284e-01,9.605551e-01,3.011983e-02,2.240643e-02
min,0.000000e+00,0.000000e+00,-2.017497e+04,-7.375435e-02,-9.848880e+03,-3.434176e+04,-8.551914e+03,-2.344957e+00,-1.043737e+13,-1.077101e+03,...,-1.687572e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-5.250904e-01,-4.449529e-01,-1.000000e+00,-2.000000e+00,-1.265686e-01,-8.609413e-02
25%,5.500000e+02,5.040000e+02,-1.449710e-01,-2.956479e-02,-5.967524e-02,-1.655826e-01,-1.057050e-01,-1.996543e-01,-1.960470e-01,-2.280967e-01,...,-4.050297e-01,-4.651562e-04,-1.992532e-04,-2.203252e-05,-1.521701e-01,-7.377038e-02,-3.887695e-15,-2.000000e+00,-1.998819e-02,-9.561389e-03
50%,1.098000e+03,9.560000e+02,-8.368272e-04,5.523058e-03,2.109505e-02,2.475614e-03,1.175234e-02,-4.064488e-02,-7.395084e-03,-3.029069e-02,...,-8.502064e-02,-3.951567e-12,-1.418487e-13,-1.591224e-16,-1.476793e-02,9.782702e-05,0.000000e+00,-6.597540e-01,1.117279e-05,-1.570681e-04
75%,1.657000e+03,1.401000e+03,1.199108e-01,1.078554e-01,1.952209e-01,3.037236e-01,1.556464e-01,1.303819e-01,1.832071e-01,1.764751e-01,...,1.909600e-01,-5.219879e-40,0.000000e+00,0.000000e+00,1.772415e-01,7.855728e-02,0.000000e+00,-5.188884e-08,2.047074e-02,9.520990e-03
max,2.158000e+03,1.812000e+03,3.252527e+03,1.068448e+16,3.823001e+03,1.239737e+03,6.785965e+04,1.378195e+00,5.203165e+02,7.677125e+01,...,4.957758e+01,0.000000e+00,0.000000e+00,0.000000e+00,1.569265e+00,6.844833e-01,1.000000e+00,0.000000e+00,1.435858e-01,9.349781e-02


In [7]:
# explore small portion of dataset
# timestamp 0 and 1
ndf = df[['id','timestamp','derived_0','derived_1','derived_2','derived_3','derived_4','y']]
ndf.describe()

,id,timestamp,derived_0,derived_1,derived_2,derived_3,derived_4,y
count,1.710756e+06,1.710756e+06,1.637797e+06,1.629727e+06,1.312105e+06,1.561285e+06,1.304298e+06,1.710756e+06
mean,1.093858e+03,9.456257e+02,-4.536046e+00,7.729436e+11,-3.320328e-01,-5.046012e-01,1.801661e+01,2.217509e-04
std,6.308563e+02,5.195685e+02,2.497382e+02,7.620606e+13,6.519810e+01,1.020749e+02,9.258360e+02,2.240643e-02
min,0.000000e+00,0.000000e+00,-2.017497e+04,-7.375435e-02,-9.848880e+03,-3.434176e+04,-8.551914e+03,-8.609413e-02
25%,5.500000e+02,5.040000e+02,-1.449710e-01,-2.956479e-02,-5.967524e-02,-1.655826e-01,-1.057050e-01,-9.561389e-03
50%,1.098000e+03,9.560000e+02,-8.368272e-04,5.523058e-03,2.109505e-02,2.475614e-03,1.175234e-02,-1.570681e-04
75%,1.657000e+03,1.401000e+03,1.199108e-01,1.078554e-01,1.952209e-01,3.037236e-01,1.556464e-01,9.520990e-03
max,2.158000e+03,1.812000e+03,3.252527e+03,1.068448e+16,3.823001e+03,1.239737e+03,6.785965e+04,9.349781e-02


In [42]:
#ndf.loc[ndf.id==10]
print "number of times security {} is traded: {}".format(10,sum(ndf.id==10))

number of times security 10 is traded: 116


In [47]:
print "number of securities traded at timestamp {} are : {}".format(0,sum(ndf.timestamp==0))
print "number of securities traded at timestamp {} are : {}".format(1,sum(ndf.timestamp==1))
print "number of securities traded at timestamp {} are : {}".format(25,sum(ndf.timestamp==25))

number of securities traded at timestamp 0 are : 750
number of securities traded at timestamp 1 are : 750
number of securities traded at timestamp 25 are : 745


In [45]:
#list securities at timestamp 0
ndf.loc[ndf.timestamp==0]

,id,timestamp,derived_0,derived_1,derived_2,derived_3,derived_4,y
0,10,0,0.370326,-0.006316,0.222831,-0.213030,0.729277,-0.011753
1,11,0,0.014765,-0.038064,-0.017425,0.320652,-0.034134,-0.001240
2,12,0,-0.010622,-0.050577,3.379575,-0.157525,-0.068550,-0.020940
3,25,0,NaN,NaN,NaN,NaN,NaN,-0.015959
4,26,0,0.176693,-0.025284,-0.057680,0.015100,0.180894,-0.007338
5,27,0,0.346856,0.166239,-6.080701,-0.992249,-0.125916,0.031425
6,31,0,0.072036,0.014931,NaN,0.014063,NaN,-0.032895
7,38,0,0.300062,0.071251,-0.074451,-0.065292,-0.011286,0.015803
8,39,0,-0.003511,-0.034270,0.082372,-0.023937,-0.025750,-0.027593
9,40,0,-0.083330,0.081935,-2.048438,-0.206856,-0.839563,0.006662


In [46]:
#list all rows where security 10 data is available in train data 
ndf.loc[ndf.id==10]

,id,timestamp,derived_0,derived_1,derived_2,derived_3,derived_4,y
0,10,0,0.370326,-0.006316,0.222831,-0.213030,0.729277,-0.011753
750,10,1,0.370326,-0.008504,0.224193,-0.216165,0.729277,0.005850
1500,10,2,0.370326,-0.009375,0.224736,-0.217415,0.729277,-0.000476
2250,10,3,0.370326,-0.010127,0.225204,-0.218492,0.729277,0.005212
3000,10,4,0.370326,-0.011338,0.225958,-0.220228,0.729277,-0.077211
3750,10,5,0.370326,-0.011824,0.226261,-0.220925,0.729277,0.006086
4500,10,6,0.370326,-0.012245,0.226523,-0.221528,0.729277,-0.027747
5250,10,7,0.370326,-0.012610,0.226750,-0.222051,0.729277,-0.033304
6000,10,8,0.370326,-0.012926,0.226947,-0.222505,0.729277,-0.002235
6750,10,9,0.370326,-0.013200,0.227118,-0.222898,0.729277,0.067070


In [17]:
range(10)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [27]:
df[['id', 'timestamp']].hist()

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x000000000C7B7518>,
        <matplotlib.axes._subplots.AxesSubplot object at 0x000000000C7E6F28>]], dtype=object)

In [ ]:
df.plot()